In [1]:
#Dev Babbar 20BAI10119
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/content/drug200.csv")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [4]:
df["Sex"].replace({"F":1,"M":0},inplace=True)
df["Cholesterol"].replace({"HIGH":1,"NORMAL":0},inplace=True)
df["BP"].replace({"HIGH":2,"NORMAL":0,"LOW":1},inplace=True)
df["Drug"].replace({"DrugY":0,"drugC":1,"drugX":2,"drugA":3,"drugB":4},inplace=True)

In [5]:
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,1,2,1,25.355,0
1,47,0,1,1,13.093,1
2,47,0,1,1,10.114,1
3,28,1,0,1,7.798,2
4,61,1,1,1,18.043,0


In [6]:
x=df.iloc[:,:-1].values 
y=df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[("encoder",OneHotEncoder(),[2])],remainder="passthrough")
x = np.array(ct.fit_transform(np.array(x)))

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [9]:
#Scaling data
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

In [10]:
print(x_train[0])

[-0.68402479  1.55056613 -0.79539491 -0.83405574  1.03823026 -1.1055416
 -0.74045911]


In [11]:
import tensorflow as tf
# Adjusting dimension of y_train
from keras.utils import np_utils
y_train_ann = np_utils.to_categorical(y_train)
y_test_ann = np_utils.to_categorical(y_test)

In [12]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    # Check accuracy
    if (logs.get('accuracy') == 1.0):
      # Stop if threshold is met
      print("\nAccuracy reached 100%\nStopping")
      self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

In [13]:
print(x_train)

[[-0.68402479  1.55056613 -0.79539491 ...  1.03823026 -1.1055416
  -0.74045911]
 [ 1.46193533 -0.64492573 -0.79539491 ... -0.96317747  0.90453403
  -0.87126694]
 [ 1.46193533 -0.64492573 -0.79539491 ...  1.03823026 -1.1055416
  -0.89684776]
 ...
 [-0.68402479 -0.64492573  1.25723711 ...  1.03823026  0.90453403
   1.83283243]
 [ 1.46193533 -0.64492573 -0.79539491 ...  1.03823026  0.90453403
  -0.04203377]
 [-0.68402479  1.55056613 -0.79539491 ...  1.03823026  0.90453403
  -0.42422495]]


In [14]:
from tensorflow.keras import regularizers

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=7, activation="relu"))
model.add(tf.keras.layers.Dense(units=7, activation="relu"))
model.add(tf.keras.layers.Dense(units=7, activation="relu"))
model.add(tf.keras.layers.Dense(units=7, activation="relu"))
model.add(tf.keras.layers.Dense(units=5, activation="softmax"))

In [15]:
#Training our model
Optimizer = 'adam'
Loss = "categorical_crossentropy"

model.compile(optimizer=Optimizer, loss=Loss, metrics=["accuracy"])

model.fit(x_train,y_train_ann,batch_size=32,epochs=2000,validation_data = (x_test, y_test_ann), callbacks = [callbacks])

Epoch 1/2000
5/5 [==============================] - 3s 136ms/step - loss: 1.6266 - accuracy: 0.1312 - val_loss: 1.6285 - val_accuracy: 0.1500
Epoch 2/2000
5/5 [==============================] - 0s 16ms/step - loss: 1.6106 - accuracy: 0.2062 - val_loss: 1.6138 - val_accuracy: 0.2500
Epoch 3/2000
5/5 [==============================] - 0s 15ms/step - loss: 1.5964 - accuracy: 0.3562 - val_loss: 1.6001 - val_accuracy: 0.4000
Epoch 4/2000
5/5 [==============================] - 0s 20ms/step - loss: 1.5834 - accuracy: 0.3938 - val_loss: 1.5879 - val_accuracy: 0.4000
Epoch 5/2000
5/5 [==============================] - 0s 37ms/step - loss: 1.5703 - accuracy: 0.4187 - val_loss: 1.5747 - val_accuracy: 0.3750
Epoch 6/2000
5/5 [==============================] - 0s 33ms/step - loss: 1.5573 - accuracy: 0.4375 - val_loss: 1.5620 - val_accuracy: 0.3750
Epoch 7/2000
5/5 [==============================] - 0s 24ms/step - loss: 1.5442 - accuracy: 0.4812 - val_loss: 1.5493 - val_accuracy: 0.4000
Epoch 8/2000

In [16]:
# [loss, accuracy]
eval = model.evaluate(x_test,y_test_ann)
print("Accuracy",eval[1])

2/2 [==============================] - 0s 7ms/step - loss: 0.0768 - accuracy: 1.0000
Accuracy 1.0


In [17]:
# Generate random input samples
num_samples = 10
age = np.random.randint(18, 80, num_samples)
sex = np.random.choice(['F', 'M'], num_samples)
bp = np.random.choice(['HIGH', 'NORMAL', 'LOW'], num_samples)
cholesterol = np.random.choice(['HIGH', 'NORMAL'], num_samples)
na_to_k = np.random.uniform(0.1, 1.5, num_samples)

In [18]:
# Prepare input data
input_data = pd.DataFrame({'Age': age, 'Sex': sex, 'BP': bp, 'Cholesterol': cholesterol, 'Na_to_K': na_to_k})

In [19]:
# Replace categorical values with numerical representations
input_data["Sex"].replace({"F": 1, "M": 0}, inplace=True)
input_data["Cholesterol"].replace({"HIGH": 1, "NORMAL": 0}, inplace=True)
input_data["BP"].replace({"HIGH": 2, "NORMAL": 0, "LOW": 1}, inplace=True)

In [20]:
# Perform one-hot encoding for the selected categorical columns
ct = ColumnTransformer(transformers=[("encoder",OneHotEncoder(),[2])],remainder="passthrough")
input_data = np.array(ct.fit_transform(np.array(input_data)))

In [21]:
# Scale the input data
scaler = StandardScaler()
input_data = scaler.fit_transform(input_data)

In [22]:
print(input_data)

[[-0.65465367  1.22474487 -0.65465367 -0.25264558  0.81649658 -0.81649658
   0.25106814]
 [-0.65465367  1.22474487 -0.65465367 -1.17901269  0.81649658  1.22474487
  -1.23920474]
 [-0.65465367  1.22474487 -0.65465367  0.25264558 -1.22474487 -0.81649658
  -0.18598466]
 [ 1.52752523 -0.81649658 -0.65465367 -0.67372154  0.81649658 -0.81649658
  -0.16655521]
 [-0.65465367  1.22474487 -0.65465367 -0.08421519 -1.22474487  1.22474487
  -0.84565357]
 [ 1.52752523 -0.81649658 -0.65465367  0.50529115  0.81649658  1.22474487
  -0.92749257]
 [-0.65465367 -0.81649658  1.52752523  1.43165827  0.81649658 -0.81649658
  -0.89438473]
 [ 1.52752523 -0.81649658 -0.65465367 -0.75793673 -1.22474487 -0.81649658
   0.69966883]
 [-0.65465367 -0.81649658  1.52752523  1.93694942  0.81649658 -0.81649658
   1.67616712]
 [-0.65465367 -0.81649658  1.52752523 -1.17901269 -1.22474487  1.22474487
   1.63237139]]


In [23]:
# Scale the input data
scaler = StandardScaler()
input_data = scaler.fit_transform(input_data)

In [24]:
# Make predictions
predictions = model.predict(input_data)

# Reverse categorical encoding for drug labels
drug_mapping = {0: "DrugY", 1: "drugC", 2: "drugX", 3: "drugA", 4: "drugB"}
predicted_drugs = [drug_mapping[np.argmax(pred)] for pred in predictions]

# Print results
for i in range(num_samples):
    print("Input:", input_data[i])
    print("Predicted Drug:", predicted_drugs[i])

1/1 [==============================] - 0s 109ms/step
Input: [-0.65465367  1.22474487 -0.65465367 -0.25264558  0.81649658 -0.81649658
  0.25106814]
Predicted Drug: DrugY
Input: [-0.65465367  1.22474487 -0.65465367 -1.17901269  0.81649658  1.22474487
 -1.23920474]
Predicted Drug: drugB
Input: [-0.65465367  1.22474487 -0.65465367  0.25264558 -1.22474487 -0.81649658
 -0.18598466]
Predicted Drug: drugX
Input: [ 1.52752523 -0.81649658 -0.65465367 -0.67372154  0.81649658 -0.81649658
 -0.16655521]
Predicted Drug: drugX
Input: [-0.65465367  1.22474487 -0.65465367 -0.08421519 -1.22474487  1.22474487
 -0.84565357]
Predicted Drug: drugC
Input: [ 1.52752523 -0.81649658 -0.65465367  0.50529115  0.81649658  1.22474487
 -0.92749257]
Predicted Drug: drugX
Input: [-0.65465367 -0.81649658  1.52752523  1.43165827  0.81649658 -0.81649658
 -0.89438473]
Predicted Drug: drugB
Input: [ 1.52752523 -0.81649658 -0.65465367 -0.75793673 -1.22474487 -0.81649658
  0.69966883]
Predicted Drug: DrugY
Input: [-0.65465367